# Phase 2: Data Preprocessing

## Overview
This notebook implements a comprehensive data preprocessing pipeline for the CDC Diabetes Health Indicators dataset (BRFSS 2015). Key processes include profile deduplication with frequency-based sample weights, logical consistency validation, target variable binarization, and weight normalization.

**Core Objectives:**
- **Profile Aggregation**: Merge identical clinical feature combinations to reduce redundancy
- **Quality Control**: Remove logically inconsistent records
- **Target Re-alignment**: Convert trinary diabetes status (0/1/2) to binary classification target (0/1)
- **Weight Preservation**: Maintain statistical representativeness through normalized sample weights (mean=1.0)
- **Integrity Assurance**: Ensure zero duplicates, zero missing values, and valid target encoding

**Pipeline Stages:**
1. Environment Configuration
2. Path Configuration
3. Utility Functions
4. Data Ingestion
5. Memory Optimization
6. Profile Aggregation
7. Aggregation Verification
8. Logical Consistency Cleaning
9. Target Re-alignment (Binarization)
10. Feature Cleanup
11. Ultimate Deduplication (Critical)
12. Weight Normalization
13. Post-Normalization Verification
14. Export Final Dataset
15. Preprocessing Pipeline Summary

## Output Dataset
**File**: `data/processed/data_preprocessing/CDC_Diabetes_Cleaned.csv`

**Purpose**: Serves as the foundation dataset for downstream feature engineering (Phase 3). Each record represents a unique clinical profile with statistically representative weight, eliminating redundant survey responses while preserving population-level prevalence ratios.

**Schema**:
- **21 Original Features**: HighBP, HighChol, CholCheck, BMI, Smoker, Stroke, HeartDiseaseorAttack, PhysActivity, Fruits, Veggies, HvyAlcoholConsump, AnyHealthcare, NoDocbcCost, GenHlth, MentHlth, PhysHlth, DiffWalk, Sex, Age, Education, Income
- **Diabetes_binary** (Target): Binary classification label (0=Non-diabetic, 1=Pre-diabetic or Diabetic)
- **Sample_Weight**: Normalized frequency weight (mean=1.0) for weighted model training

---
## 1. Environment Configuration

In [27]:
# Standard library imports
import os
import sys
from pathlib import Path
from typing import Tuple

# Third-party imports
import pandas as pd
import numpy as np

# Display configuration
pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', '{:.4f}'.format)

print(f"Python version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Python version: 3.13.11 | packaged by Anaconda, Inc. | (main, Dec 10 2025, 21:21:58) [MSC v.1929 64 bit (AMD64)]
Pandas version: 3.0.0
NumPy version: 2.3.5


---
## 2. Path Configuration

In [28]:
# Define project directory structure
PROJECT_ROOT = Path.cwd()
DATA_RAW_DIR = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
PREPROCESSING_OUTPUT_DIR = DATA_PROCESSED_DIR / "data_preprocessing"

# Source and destination file paths
INPUT_FILE = DATA_RAW_DIR / "CDC Diabetes Dataset.csv"
OUTPUT_FILE = PREPROCESSING_OUTPUT_DIR / "CDC_Diabetes_Cleaned.csv"

# Create output directory if it doesn't exist
PREPROCESSING_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"[CONFIG] Output file: {OUTPUT_FILE}")

print(f"[CONFIG] Project root: {PROJECT_ROOT}")
print(f"[CONFIG] Preprocessing output directory: {PREPROCESSING_OUTPUT_DIR}")
print(f"[CONFIG] Input file: {INPUT_FILE}")

[CONFIG] Output file: D:\ProgramSoftware\PyCharm\WorkPlace\DiaMetric-CDC\data\processed\data_preprocessing\CDC_Diabetes_Cleaned.csv
[CONFIG] Project root: D:\ProgramSoftware\PyCharm\WorkPlace\DiaMetric-CDC
[CONFIG] Preprocessing output directory: D:\ProgramSoftware\PyCharm\WorkPlace\DiaMetric-CDC\data\processed\data_preprocessing
[CONFIG] Input file: D:\ProgramSoftware\PyCharm\WorkPlace\DiaMetric-CDC\data\raw\CDC Diabetes Dataset.csv


---
## 3. Utility Functions

In [29]:
def load_dataset(filepath: Path) -> pd.DataFrame:
    """
    Load CSV dataset with error handling.
    
    Parameters
    ----------
    filepath : Path
        Path to the CSV file.
    
    Returns
    -------
    pd.DataFrame
        Loaded dataframe.
    
    Raises
    ------
    FileNotFoundError
        If file does not exist.
    ValueError
        If file is empty.
    """
    if not filepath.exists():
        raise FileNotFoundError(f"Dataset not found at: {filepath}")
    
    df = pd.read_csv(filepath)
    
    if df.empty:
        raise ValueError(f"Dataset is empty: {filepath}")
    
    print(f"[SUCCESS] Loaded {len(df):,} records with {df.shape[1]} columns.")
    return df


def optimize_memory(df: pd.DataFrame) -> pd.DataFrame:
    """
    Downcast numeric columns to reduce memory footprint.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with numeric columns.
    
    Returns
    -------
    pd.DataFrame
        Memory-optimized dataframe.
    """
    initial_mem = df.memory_usage(deep=True).sum() / 1024**2
    
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    
    final_mem = df.memory_usage(deep=True).sum() / 1024**2
    reduction_pct = (1 - final_mem / initial_mem) * 100
    
    print(f"[MEMORY] {initial_mem:.2f} MB -> {final_mem:.2f} MB ({reduction_pct:.1f}% reduction)")
    return df

In [30]:
def aggregate_profiles(df: pd.DataFrame) -> Tuple[pd.DataFrame, dict]:
    """
    Aggregate identical clinical profiles and compute sample weights.
    
    Groups records by feature values, treating duplicate rows as repeated
    observations of the same clinical archetype. Profile frequency stored
    as Sample_Weight.
    
    Parameters
    ----------
    df : pd.DataFrame
        Raw dataframe with potential duplicate profiles.
    
    Returns
    -------
    Tuple[pd.DataFrame, dict]
        - Aggregated dataframe with unique profiles and Sample_Weight column.
        - Summary statistics dictionary.
    """
    original_count = len(df)
    feature_columns = df.columns.tolist()
    
    # Group by all columns and compute frequency
    df_aggregated = (
        df.groupby(feature_columns, as_index=False)
        .size()
        .rename(columns={'size': 'Sample_Weight'})
    )
    
    unique_count = len(df_aggregated)
    reduction_pct = (1 - unique_count / original_count) * 100
    
    summary = {
        'original_count': original_count,
        'unique_profiles': unique_count,
        'duplicates_removed': original_count - unique_count,
        'reduction_percentage': reduction_pct,
        'weight_sum': df_aggregated['Sample_Weight'].sum()
    }
    
    return df_aggregated, summary


def verify_data_integrity(
    df: pd.DataFrame,
    original_count: int,
    expected_columns: int = 23
) -> None:
    """
    Validate data integrity constraints after aggregation.
    
    Parameters
    ----------
    df : pd.DataFrame
        Aggregated dataframe to verify.
    original_count : int
        Original row count before aggregation.
    expected_columns : int, default 23
        Expected number of columns (22 features + Sample_Weight).
    
    Raises
    ------
    AssertionError
        If any integrity constraint is violated.
    """
    # Verify column count (22 original + Sample_Weight)
    assert df.shape[1] == expected_columns, (
        f"Column count mismatch: expected {expected_columns}, got {df.shape[1]}"
    )
    
    # Verify weight summation equals original count
    weight_sum = df['Sample_Weight'].sum()
    assert weight_sum == original_count, (
        f"Weight sum mismatch: expected {original_count}, got {weight_sum}"
    )
    
    # Verify no null values in weight column
    assert df['Sample_Weight'].notna().all(), (
        "Sample_Weight contains null values"
    )
    
    # Verify all weights are positive integers
    assert (df['Sample_Weight'] > 0).all(), (
        "Sample_Weight contains non-positive values"
    )
    
    print("[VALIDATED] All data integrity assertions passed.")

In [31]:
def save_processed_data(df: pd.DataFrame, filepath: Path) -> None:
    """
    Export processed dataframe to CSV.
    
    Parameters
    ----------
    df : pd.DataFrame
        Dataframe to export.
    filepath : Path
        Destination file path.
    """
    # Ensure output directory exists
    filepath.parent.mkdir(parents=True, exist_ok=True)
    
    df.to_csv(filepath, index=False)
    
    file_size_mb = filepath.stat().st_size / 1024**2
    print(f"[SUCCESS] Saved to: {filepath}")
    print(f"[INFO] File size: {file_size_mb:.2f} MB")

---
## 4. Data Ingestion

In [32]:
# Load raw dataset
df_raw = load_dataset(INPUT_FILE)

# Display initial structure
print(f"\n[INFO] Shape: {df_raw.shape}")
print(f"[INFO] Columns: {df_raw.columns.tolist()}")
df_raw.head()

[SUCCESS] Loaded 253,680 records with 22 columns.

[INFO] Shape: (253680, 22)
[INFO] Columns: ['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income']


,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0000,1.0000,1.0000,1.0000,40.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,1.0000,0.0000,5.0000,18.0000,15.0000,1.0000,0.0000,9.0000,4.0000,3.0000
1,0.0000,0.0000,0.0000,0.0000,25.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,3.0000,0.0000,0.0000,0.0000,0.0000,7.0000,6.0000,1.0000
2,0.0000,1.0000,1.0000,1.0000,28.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,1.0000,5.0000,30.0000,30.0000,1.0000,0.0000,9.0000,4.0000,8.0000
3,0.0000,1.0000,0.0000,1.0000,27.0000,0.0000,0.0000,0.0000,1.0000,1.0000,1.0000,0.0000,1.0000,0.0000,2.0000,0.0000,0.0000,0.0000,0.0000,11.0000,3.0000,6.0000
4,0.0000,1.0000,1.0000,1.0000,24.0000,0.0000,0.0000,0.0000,1.0000,1.0000,1.0000,0.0000,1.0000,0.0000,2.0000,3.0000,0.0000,0.0000,0.0000,11.0000,5.0000,4.0000


In [33]:
# Verify expected column count
EXPECTED_RAW_COLUMNS = 22
assert df_raw.shape[1] == EXPECTED_RAW_COLUMNS, (
    f"Expected {EXPECTED_RAW_COLUMNS} columns, got {df_raw.shape[1]}"
)
print(f"[VALIDATED] Column count verified: {df_raw.shape[1]}")

[VALIDATED] Column count verified: 22


---
## 5. Memory Optimization

In [34]:
# Downcast numeric types to reduce memory footprint
df_optimized = optimize_memory(df_raw.copy())

# Display optimized dtypes
print("\n[INFO] Optimized data types:")
print(df_optimized.dtypes.value_counts())

[MEMORY] 42.58 MB -> 21.29 MB (50.0% reduction)

[INFO] Optimized data types:
float32    22
Name: count, dtype: int64


---
## 6. Profile Aggregation

In [35]:
# Execute profile convergence strategy
df_aggregated, agg_summary = aggregate_profiles(df_optimized)

# Display aggregation summary
print("\n" + "="*60)
print("PROFILE AGGREGATION SUMMARY")
print("="*60)
print(f"Original record count:    {agg_summary['original_count']:>12,}")
print(f"Unique clinical profiles: {agg_summary['unique_profiles']:>12,}")
print(f"Duplicate records merged: {agg_summary['duplicates_removed']:>12,}")
print(f"Data reduction:           {agg_summary['reduction_percentage']:>11.2f}%")
print(f"Weight sum verification:  {agg_summary['weight_sum']:>12,}")
print("="*60)


PROFILE AGGREGATION SUMMARY
Original record count:         253,680
Unique clinical profiles:      229,781
Duplicate records merged:       23,899
Data reduction:                  9.42%
Weight sum verification:       253,680


In [36]:
# Examine weight distribution
print("[INFO] Sample_Weight distribution:")
print(df_aggregated['Sample_Weight'].describe())

print(f"\n[INFO] Profiles with weight > 1: {(df_aggregated['Sample_Weight'] > 1).sum():,}")
print(f"[INFO] Maximum weight: {df_aggregated['Sample_Weight'].max():,}")

[INFO] Sample_Weight distribution:
count   229781.0000
mean         1.1040
std          0.8490
min          1.0000
25%          1.0000
50%          1.0000
75%          1.0000
max         59.0000
Name: Sample_Weight, dtype: float64

[INFO] Profiles with weight > 1: 11,187
[INFO] Maximum weight: 59


---
## 7. Aggregation Verification

In [37]:
print("\n" + "="*60)
print("STAGE 7 VERIFICATION: Profile Aggregation")
print("="*60)

# Execute integrity assertions
verify_data_integrity(
    df=df_aggregated,
    original_count=agg_summary['original_count'],
    expected_columns=23  # 22 features + Sample_Weight
)

# Verify weight statistics
assert df_aggregated['Sample_Weight'].min() > 0, "Invalid: negative or zero weights detected"
assert df_aggregated['Sample_Weight'].sum() == agg_summary['original_count'], "Weight sum mismatch"

print(f"\n[VALIDATED] Shape: {df_aggregated.shape}")
print(f"[VALIDATED] Weight range: [{df_aggregated['Sample_Weight'].min()}, {df_aggregated['Sample_Weight'].max()}]")
print(f"[VALIDATED] Weight mean: {df_aggregated['Sample_Weight'].mean():.2f}")
print("="*60)


STAGE 7 VERIFICATION: Profile Aggregation
[VALIDATED] All data integrity assertions passed.

[VALIDATED] Shape: (229781, 23)
[VALIDATED] Weight range: [1, 59]
[VALIDATED] Weight mean: 1.10


In [38]:
# Display aggregated dataframe structure
print(f"\n[INFO] Aggregated dataset preview:")
df_aggregated.head(10)


[INFO] Aggregated dataset preview:


,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Sample_Weight
0,0.0000,0.0000,0.0000,0.0000,14.0000,1.0000,0.0000,0.0000,1.0000,1.0000,1.0000,0.0000,1.0000,0.0000,3.0000,4.0000,4.0000,0.0000,1.0000,11.0000,6.0000,8.0000,1
1,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,5.0000,7.0000,1
2,0.0000,0.0000,0.0000,0.0000,15.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,5.0000,20.0000,28.0000,1.0000,0.0000,10.0000,6.0000,4.0000,1
3,0.0000,0.0000,0.0000,0.0000,15.0000,1.0000,0.0000,1.0000,0.0000,1.0000,1.0000,0.0000,0.0000,1.0000,3.0000,0.0000,29.0000,0.0000,0.0000,7.0000,5.0000,2.0000,1
4,0.0000,0.0000,0.0000,0.0000,16.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,2.0000,0.0000,0.0000,0.0000,0.0000,11.0000,5.0000,5.0000,1
5,0.0000,0.0000,0.0000,0.0000,16.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,1.0000,0.0000,2.0000,7.0000,5.0000,0.0000,0.0000,1.0000,5.0000,5.0000,1
6,0.0000,0.0000,0.0000,0.0000,16.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,1.0000,1.0000,2.0000,0.0000,0.0000,0.0000,0.0000,6.0000,5.0000,6.0000,1
7,0.0000,0.0000,0.0000,0.0000,16.0000,0.0000,0.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,2.0000,0.0000,4.0000,0.0000,0.0000,3.0000,6.0000,8.0000,1
8,0.0000,0.0000,0.0000,0.0000,16.0000,0.0000,0.0000,0.0000,1.0000,1.0000,1.0000,0.0000,0.0000,0.0000,2.0000,0.0000,0.0000,0.0000,0.0000,9.0000,5.0000,2.0000,1
9,0.0000,0.0000,0.0000,0.0000,16.0000,0.0000,0.0000,1.0000,1.0000,0.0000,1.0000,0.0000,1.0000,1.0000,4.0000,0.0000,3.0000,0.0000,0.0000,6.0000,6.0000,2.0000,1


---
## 8. Logical Consistency Cleaning

Identifies and removes records with logical inconsistencies between self-reported health indicators. Profiles reporting 30 days of poor physical health (PhysHlth=30) while claiming excellent general health (GenHlth=1) represent contradictory self-assessments that compromise model reliability.

In [39]:
def remove_logical_inconsistencies(df: pd.DataFrame) -> Tuple[pd.DataFrame, int]:
    """
    Remove records with contradictory health indicator combinations.
    
    Logical inconsistency: PhysHlth=30 with GenHlth=1.
    - PhysHlth=30: Poor physical health for all 30 days
    - GenHlth=1: Excellent general health
    
    Mutually exclusive conditions indicating response errors or
    survey question misinterpretation.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with PhysHlth and GenHlth columns.
    
    Returns
    -------
    Tuple[pd.DataFrame, int]
        Cleaned dataframe and count of removed records.
    """
    initial_count = len(df)
    
    # Define inconsistency mask: maximum physical distress + excellent health
    inconsistency_mask = (df['PhysHlth'] == 30) & (df['GenHlth'] == 1)
    
    # Filter out inconsistent records
    df_cleaned = df[~inconsistency_mask].copy()
    
    removed_count = initial_count - len(df_cleaned)
    
    return df_cleaned, removed_count

In [40]:
# Execute logical consistency cleaning
df_consistent, removed_count = remove_logical_inconsistencies(df_aggregated)

print("\n" + "="*60)
print("STAGE 8 VERIFICATION: Logical Consistency Cleaning")
print("="*60)
print(f"Records before cleaning:  {len(df_aggregated):>12,}")
print(f"Inconsistent records:     {removed_count:>12,}")
print(f"Records after cleaning:   {len(df_consistent):>12,}")

# Calculate weight statistics
weight_sum_before = df_aggregated['Sample_Weight'].sum()
weight_sum_after = df_consistent['Sample_Weight'].sum()
weight_removed = weight_sum_before - weight_sum_after

print(f"\nWeight statistics:")
print(f"  Weight sum before:      {weight_sum_before:>12,.0f}")
print(f"  Weight sum after:       {weight_sum_after:>12,.0f}")
print(f"  Weight removed:         {weight_removed:>12,.0f}")

# Verify no contradictory records remain
inconsistent_check = ((df_consistent['PhysHlth'] == 30) & (df_consistent['GenHlth'] == 1)).sum()
assert inconsistent_check == 0, f"Inconsistency removal failed: {inconsistent_check} contradictory records remain"

print(f"\n[VALIDATED] No contradictory records remaining: {inconsistent_check}")
print(f"[VALIDATED] Data integrity maintained")
print("="*60)


STAGE 8 VERIFICATION: Logical Consistency Cleaning
Records before cleaning:       229,781
Inconsistent records:              416
Records after cleaning:        229,365

Weight statistics:
  Weight sum before:           253,680
  Weight sum after:            253,264
  Weight removed:                  416

[VALIDATED] No contradictory records remaining: 0
[VALIDATED] Data integrity maintained


---
## 9. Target Re-alignment (Binarization)

The original trinary target variable (`Diabetes_012`) is transformed into a binary classification target. Pre-diabetic cases (value 1) are consolidated with diabetic cases (value 2) to create a unified positive class, enabling binary classification modeling.

In [41]:
def binarize_target(df: pd.DataFrame, source_col: str = 'Diabetes_012') -> pd.DataFrame:
    """
    Convert trinary diabetes target to binary classification target.
    
    Mapping:
    - 0 (No diabetes) -> 0 (Negative class)
    - 1 (Pre-diabetes) -> 1 (Positive class)
    - 2 (Diabetes) -> 1 (Positive class)
    
    Original column retained for validation and removed in subsequent cleanup stage.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with trinary target column.
    source_col : str, default 'Diabetes_012'
        Name of the original target column.
    
    Returns
    -------
    pd.DataFrame
        Dataframe with new binary target column 'Diabetes_binary'.
    
    Raises
    ------
    ValueError
        If source column contains NaN values.
    """
    df = df.copy()
    
    # Check for NaN values
    if df[source_col].isna().any():
        nan_count = df[source_col].isna().sum()
        raise ValueError(
            f"Found {nan_count} NaN values in {source_col}. "
            "Handle missing values before binarization."
        )
    
    # Binary mapping: 0 stays 0, any positive value becomes 1
    df['Diabetes_binary'] = df[source_col].apply(lambda x: 0 if x == 0 else 1)
    
    return df

In [42]:
# Execute target binarization on cleaned dataset
df_binarized = binarize_target(df_consistent)

print("\n" + "="*60)
print("STAGE 9 VERIFICATION: Target Re-alignment (Binarization)")
print("="*60)
print("\nOriginal target distribution (Diabetes_012):")
original_dist = df_binarized['Diabetes_012'].value_counts().sort_index()
for val, count in original_dist.items():
    label = {0: 'No diabetes', 1: 'Pre-diabetes', 2: 'Diabetes'}[int(val)]
    print(f"  {int(val)} ({label}): {count:,}")

print("\nBinarized target distribution (Diabetes_binary):")
binary_dist = df_binarized['Diabetes_binary'].value_counts().sort_index()
for val, count in binary_dist.items():
    label = {0: 'Negative', 1: 'Positive'}[val]
    print(f"  {val} ({label}): {count:,}")

# Verify mapping correctness
expected_negative = original_dist.get(0, 0)
expected_positive = original_dist.get(1, 0) + original_dist.get(2, 0)
actual_negative = binary_dist.get(0, 0)
actual_positive = binary_dist.get(1, 0)

assert expected_negative == actual_negative, "Negative class count mismatch"
assert expected_positive == actual_positive, "Positive class count mismatch"
assert set(df_binarized['Diabetes_binary'].unique()) == {0, 1}, "Invalid values in binary target"

print(f"\n[VALIDATED] Mapping correctness: Negative={actual_negative:,}, Positive={actual_positive:,}")
print(f"[VALIDATED] Binary target contains only values: {sorted(df_binarized['Diabetes_binary'].unique())}")
print(f"[VALIDATED] Class balance ratio: {actual_positive/actual_negative:.4f}")
print("="*60)


STAGE 9 VERIFICATION: Target Re-alignment (Binarization)

Original target distribution (Diabetes_012):
  0 (No diabetes): 189,697
  1 (Pre-diabetes): 4,620
  2 (Diabetes): 35,048

Binarized target distribution (Diabetes_binary):
  0 (Negative): 189,697
  1 (Positive): 39,668

[VALIDATED] Mapping correctness: Negative=189,697, Positive=39,668
[VALIDATED] Binary target contains only values: [np.int64(0), np.int64(1)]
[VALIDATED] Class balance ratio: 0.2091


---
## 10. Feature Cleanup

Removes original trinary target column to prevent target leakage. Ensures downstream modeling pipeline uses only the binary target variable.

In [43]:
# Drop original trinary target to prevent leakage
df_cleaned = df_binarized.drop(columns=['Diabetes_012'])

print("\n" + "="*60)
print("STAGE 10 VERIFICATION: Feature Cleanup")
print("="*60)

assert 'Diabetes_012' not in df_cleaned.columns, "Target leakage: Diabetes_012 column still exists"
assert 'Diabetes_binary' in df_cleaned.columns, "Binary target column missing"

print(f"[VALIDATED] Diabetes_012 removed (prevent target leakage)")
print(f"[VALIDATED] Diabetes_binary preserved (binary target)")
print(f"[VALIDATED] Current shape: {df_cleaned.shape}")
print(f"[VALIDATED] Current columns: {df_cleaned.shape[1]}")
print("="*60)


STAGE 10 VERIFICATION: Feature Cleanup
[VALIDATED] Diabetes_012 removed (prevent target leakage)
[VALIDATED] Diabetes_binary preserved (binary target)
[VALIDATED] Current shape: (229365, 23)
[VALIDATED] Current columns: 23


In [44]:
def remove_constant_features(df: pd.DataFrame, exclude: list = ['Sample_Weight']) -> Tuple[pd.DataFrame, list]:
    """
    Identify and remove features that have the same value across all samples.
    """
    constant_cols = [col for col in df.columns 
                     if df[col].nunique() <= 1 and col not in exclude]
    
    if constant_cols:
        df = df.drop(columns=constant_cols)
        print(f"[CLEANUP] Removed constant features: {constant_cols}")
    else:
        print("[CLEANUP] No constant features detected.")
    
    return df, constant_cols
print("\n" + "="*60)
print("ADDITIONAL AUDIT: Constant Feature Check")
print("="*60)

# Execute function
df_cleaned, removed_const = remove_constant_features(df_cleaned)

# Validate returned list
assert isinstance(removed_const, list), "Returned value type error"
print(f"[VALIDATED] Audit process recorded.")
print("="*60)


ADDITIONAL AUDIT: Constant Feature Check
[CLEANUP] No constant features detected.
[VALIDATED] Audit process recorded.


---
## 11. Ultimate Deduplication (Critical)

After feature cleanup (removing `Diabetes_012`), rows that originally had different `Diabetes_012` values but the same features and `Diabetes_binary` value now become duplicates. This step performs the **ultimate aggregation** by grouping all identical feature profiles and summing their weights.

In [45]:
# Perform ultimate deduplication: merge all identical profiles
print("\n" + "="*60)
print("STAGE 11: ULTIMATE DEDUPLICATION")
print("="*60)

records_before = len(df_cleaned)
feature_cols = [col for col in df_cleaned.columns if col != 'Sample_Weight']

# Check for potential duplicates
dup_mask = df_cleaned[feature_cols].duplicated(keep=False)
print(f"Records before deduplication: {records_before:,}")
print(f"Potentially duplicate profiles: {dup_mask.sum()}")

# Aggregate: group by all features, sum weights
df_deduplicated = (
    df_cleaned.groupby(feature_cols, as_index=False)
    .agg({'Sample_Weight': 'sum'})
)

records_after = len(df_deduplicated)
merged_count = records_before - records_after

print(f"Records after deduplication:  {records_after:,}")
print(f"Duplicate records merged:     {merged_count}")

# Verify no duplicates remain
final_dup_check = df_deduplicated[feature_cols].duplicated().sum()
assert final_dup_check == 0, f"Deduplication failed: {final_dup_check} duplicates remain"

# Verify weight integrity
weight_sum_before = df_cleaned['Sample_Weight'].sum()
weight_sum_after = df_deduplicated['Sample_Weight'].sum()
assert np.isclose(weight_sum_before, weight_sum_after, rtol=1e-9), "Weight sum changed during deduplication"

print(f"\n[VALIDATED] No duplicate profiles remain: {final_dup_check}")
print(f"[VALIDATED] Weight sum preserved: {weight_sum_after:,.2f}")
print(f"[VALIDATED] Dataset ready for normalization")
print("="*60)


STAGE 11: ULTIMATE DEDUPLICATION
Records before deduplication: 229,365
Potentially duplicate profiles: 138
Records after deduplication:  229,296
Duplicate records merged:     69

[VALIDATED] No duplicate profiles remain: 0
[VALIDATED] Weight sum preserved: 253,264.00
[VALIDATED] Dataset ready for normalization


---
## 12. Weight Normalization

Sample weights are normalized by dividing each weight by the mean, ensuring a distribution mean of 1.0. This facilitates stable numerical computations in downstream modeling while preserving relative prevalence ratios.

In [46]:
def normalize_weights(df: pd.DataFrame, weight_col: str = 'Sample_Weight') -> pd.DataFrame:
    """
    Normalize sample weights by dividing by mean.
    
    Ensures weight distribution has mean 1.0, facilitating stable numerical
    computations while preserving relative prevalence ratios.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with raw sample weights.
    weight_col : str, default 'Sample_Weight'
        Name of the weight column.
    
    Returns
    -------
    pd.DataFrame
        Dataframe with normalized weights.
    """
    df = df.copy()
    
    mean_weight = df[weight_col].mean()
    df[weight_col] = df[weight_col] / mean_weight
    
    return df

In [47]:
# Execute weight normalization on deduplicated dataset
pre_norm_mean = df_deduplicated['Sample_Weight'].mean()
df_normalized = normalize_weights(df_deduplicated)
post_norm_mean = df_normalized['Sample_Weight'].mean()

print("\n" + "="*60)
print("STAGE 12 VERIFICATION: Weight Normalization")
print("="*60)
print(f"Pre-normalization mean:   {pre_norm_mean:.6f}")
print(f"Post-normalization mean:  {post_norm_mean:.6f}")

# Validate mean equals 1.0 (within floating-point tolerance)
assert np.isclose(post_norm_mean, 1.0, atol=1e-10), (
    f"Weight normalization failed: mean = {post_norm_mean}, expected 1.0"
)

weight_std = df_normalized['Sample_Weight'].std()
weight_min = df_normalized['Sample_Weight'].min()
weight_max = df_normalized['Sample_Weight'].max()

print(f"\n[VALIDATED] Weight mean: {post_norm_mean:.6f} (target: 1.0)")
print(f"[VALIDATED] Weight std: {weight_std:.4f}")
print(f"[VALIDATED] Weight range: [{weight_min:.4f}, {weight_max:.4f}]")
print(f"\nWeight distribution:")
print(df_normalized['Sample_Weight'].describe())
print("="*60)


STAGE 12 VERIFICATION: Weight Normalization
Pre-normalization mean:   1.104529
Post-normalization mean:  1.000000

[VALIDATED] Weight mean: 1.000000 (target: 1.0)
[VALIDATED] Weight std: 0.7697
[VALIDATED] Weight range: [0.9054, 53.4165]

Weight distribution:
count   229296.0000
mean         1.0000
std          0.7697
min          0.9054
25%          0.9054
50%          0.9054
75%          0.9054
max         53.4165
Name: Sample_Weight, dtype: float64


---
## 13. Post-Normalization Verification

Verifies absence of duplicate rows after normalization. Although normalization operates via scalar division and should not generate duplicates, this verification ensures data integrity.

In [48]:
# Final duplicate check after normalization
print("\n" + "="*60)
print("STAGE 13 VERIFICATION: Post-Normalization Duplicate Check")
print("="*60)

df_final = df_normalized.copy()

# Check for complete duplicates (all columns including Sample_Weight)
full_dup = df_final.duplicated().sum()
print(f"Complete duplicate rows: {full_dup}")

# Check for feature duplicates (excluding Sample_Weight)
feature_cols = [col for col in df_final.columns if col != 'Sample_Weight']
feature_dup = df_final[feature_cols].duplicated().sum()
print(f"Feature duplicate rows (excluding Sample_Weight): {feature_dup}")

assert full_dup == 0, f"ERROR: {full_dup} complete duplicates found after normalization!"
assert feature_dup == 0, f"ERROR: {feature_dup} feature duplicates found after normalization!"

print(f"\n[VALIDATED] No duplicate rows detected")
print(f"[VALIDATED] Final dataset shape: {df_final.shape}")
print(f"[VALIDATED] Final record count: {len(df_final):,}")
print(f"[VALIDATED] Data pipeline complete - ready for export")
print("="*60)


STAGE 13 VERIFICATION: Post-Normalization Duplicate Check
Complete duplicate rows: 0
Feature duplicate rows (excluding Sample_Weight): 0

[VALIDATED] No duplicate rows detected
[VALIDATED] Final dataset shape: (229296, 23)
[VALIDATED] Final record count: 229,296
[VALIDATED] Data pipeline complete - ready for export


In [49]:
def final_quality_audit(df: pd.DataFrame):
    """
    Perform final data integrity checks before export.
    """
    # 1. Check for missing values
    assert df.isna().sum().sum() == 0, "Unexpected missing value found!"
    
    # 2. Check if the binary target is complete
    assert set(df['Diabetes_binary'].unique()) == {0, 1}, "Target variable does not conform to binary logic"
    
    # 3. Check if the sample weights are normalized (double check)
    assert np.isclose(df['Sample_Weight'].mean(), 1.0, atol=1e-5), "Sample weights normalization deviates from threshold"
    
    print("Data flow integrity: 100% passed.")

final_quality_audit(df_final)

Data flow integrity: 100% passed.


---
## 14. Export Final Dataset

In [50]:
# Save final cleaned and processed dataset
print("\n" + "="*60)
print("FINAL EXPORT")
print("="*60)
save_processed_data(df_final, OUTPUT_FILE)
print(f"[SUCCESS] Dataset ready for feature engineering")
print("="*60)


FINAL EXPORT
[SUCCESS] Saved to: D:\ProgramSoftware\PyCharm\WorkPlace\DiaMetric-CDC\data\processed\data_preprocessing\CDC_Diabetes_Cleaned.csv
[INFO] File size: 23.54 MB
[SUCCESS] Dataset ready for feature engineering


---
## 15. Preprocessing Pipeline Summary

In [51]:


# Comprehensive preprocessing summary
print("\n" + "#"*60)
print("DATA PREPROCESSING PIPELINE SUMMARY")
print("#"*60)

# Verify all required variables exist (use globals() for notebook scope)
required_vars = {
    'agg_summary': 'Section 6-7 (Profile Aggregation)',
    'removed_count': 'Section 8 (Logical Consistency)',
    'records_before': 'Section 11 (Ultimate Deduplication)',
    'merged_count': 'Section 11 (Ultimate Deduplication)',
    'records_after': 'Section 11 (Ultimate Deduplication)',
    'pre_norm_mean': 'Section 12 (Weight Normalization)',
    'post_norm_mean': 'Section 12 (Weight Normalization)',
    'full_dup': 'Section 13 (Post-Norm Verification)',
    'feature_dup': 'Section 13 (Post-Norm Verification)',
    'df_final': 'Section 13 (Post-Norm Verification)'
}

missing_vars = {var: stage for var, stage in required_vars.items() if var not in globals()}
if missing_vars:
    print("\nWARNING: Cannot generate complete summary")
    print("The following variables are missing:\n")
    for var, stage in missing_vars.items():
        print(f"  - {var} (from {stage})")
    print("\nExecute all cells sequentially from the beginning.")
    print("#"*60)
else:
    # Recalculate class distribution from final dataset for accuracy
    actual_negative = (df_final['Diabetes_binary'] == 0).sum()
    actual_positive = (df_final['Diabetes_binary'] == 1).sum()
    
    print("\n[INPUT]")
    print(f"  Source file: {INPUT_FILE.name}")
    print(f"  Raw records: {agg_summary['original_count']:,}")

    print("\n[PROCESSING STAGES]")
    print(f"  Stage 1: Environment Configuration")
    print(f"    - Python, Pandas, NumPy version check")
    print(f"    - Display options configured")
    print(f"    [VALIDATED] Runtime environment initialized")
    
    print(f"\n  Stage 2: Path Configuration")
    print(f"    - Project root: {PROJECT_ROOT.name}")
    print(f"    - Input: {INPUT_FILE.name}")
    print(f"    - Output: {OUTPUT_FILE.name}")
    print(f"    [VALIDATED] File paths configured")
    
    print(f"\n  Stage 3: Utility Functions")
    print(f"    - load_dataset(), optimize_memory(), aggregate_profiles()")
    print(f"    - verify_data_integrity(), save_processed_data()")
    print(f"    [VALIDATED] Helper functions defined")
    
    print(f"\n  Stage 4: Data Ingestion")
    print(f"    - Loaded: {INPUT_FILE.name}")
    print(f"    - Expected columns: 22")
    print(f"    [VALIDATED] Dataset loaded successfully")
    
    print(f"\n  Stage 5: Memory Optimization")
    print(f"    - Downcast numeric types to int8/int16")
    print(f"    - Memory footprint reduced")
    print(f"    [VALIDATED] Data types optimized")
    
    print(f"\n  Stage 6-7: Profile Aggregation & Verification")
    print(f"    - Unique profiles: {agg_summary['unique_profiles']:,}")
    print(f"    - Duplicates merged: {agg_summary['duplicates_removed']:,}")
    print(f"    - Data reduction: {agg_summary['reduction_percentage']:.2f}%")
    print(f"    - Added: Sample_Weight column")
    print(f"    [VALIDATED] Weight sum = {agg_summary['weight_sum']:,}")

    print(f"\n  Stage 8: Logical Consistency Cleaning")
    print(f"    - Inconsistent records removed: {removed_count:,}")
    print(f"    - Criteria: PhysHlth=30 & GenHlth=1 (contradictory)")
    if 'weight_removed' in globals():
        print(f"    - Weight removed: {weight_removed:,.0f}")
    print(f"    [VALIDATED] No contradictory records remain")

    print(f"\n  Stage 9: Target Re-alignment (Binarization)")
    print(f"    - Original: Diabetes_012 (0=No, 1=Pre, 2=Yes)")
    print(f"    - Binarized: Diabetes_binary (0=Negative, 1=Positive)")
    print(f"    [VALIDATED] Mapping correctness verified")

    print(f"\n  Stage 10: Feature Cleanup")
    print(f"    - Dropped: Diabetes_012 (prevent target leakage)")
    print(f"    - Retained: Diabetes_binary (binary target)")
    print(f"    [VALIDATED] No data leakage")

    print(f"\n  Stage 11: Ultimate Deduplication (Critical)")
    print(f"    - Records before: {records_before:,}")
    print(f"    - Duplicates merged: {merged_count}")
    print(f"    - Records after: {records_after:,}")
    print(f"    [VALIDATED] No duplicate profiles remain")

    print(f"\n  Stage 12: Weight Normalization")
    print(f"    - Pre-normalization mean: {pre_norm_mean:.4f}")
    print(f"    - Post-normalization mean: {post_norm_mean:.6f}")
    print(f"    [VALIDATED] Mean = 1.0, numerical stability ensured")

    print(f"\n  Stage 13: Post-Normalization Verification")
    print(f"    - Complete duplicates: {full_dup}")
    print(f"    - Feature duplicates: {feature_dup}")
    print(f"    [VALIDATED] Zero duplicates after normalization")
    
    print(f"\n  Stage 14: Export Final Dataset")
    print(f"    - Destination: {OUTPUT_FILE.name}")
    print(f"    - File saved successfully")
    print(f"    [VALIDATED] Export completed")

    print("\n[OUTPUT]")
    print(f"  File: {OUTPUT_FILE.name}")
    print(f"  Final shape: {df_final.shape}")
    print(f"  Composition: 21 features + Diabetes_binary (target) + Sample_Weight")
    print(f"  Records: {len(df_final):,}")
    print(f"  Class distribution:")
    print(f"    - Negative class: {actual_negative:,}")
    print(f"    - Positive class: {actual_positive:,}")
    print(f"    - Balance ratio: {actual_positive/actual_negative:.4f}")

    print("\n[STATUS]")
    print(f"  [VALIDATED] All validation checks passed")
    print(f"  [VALIDATED] Dataset ready for feature engineering (Phase 3)")
    print("\n" + "#"*60)


############################################################
DATA PREPROCESSING PIPELINE SUMMARY
############################################################

[INPUT]
  Source file: CDC Diabetes Dataset.csv
  Raw records: 253,680

[PROCESSING STAGES]
  Stage 1: Environment Configuration
    - Python, Pandas, NumPy version check
    - Display options configured
    [VALIDATED] Runtime environment initialized

  Stage 2: Path Configuration
    - Project root: DiaMetric-CDC
    - Input: CDC Diabetes Dataset.csv
    - Output: CDC_Diabetes_Cleaned.csv
    [VALIDATED] File paths configured

  Stage 3: Utility Functions
    - load_dataset(), optimize_memory(), aggregate_profiles()
    - verify_data_integrity(), save_processed_data()
    [VALIDATED] Helper functions defined

  Stage 4: Data Ingestion
    - Loaded: CDC Diabetes Dataset.csv
    - Expected columns: 22
    [VALIDATED] Dataset loaded successfully

  Stage 5: Memory Optimization
    - Downcast numeric types to int8/int16
    - Memo